In [1]:
import numpy as np
import pandas as pd
import scipy

import string
import re
import gc

In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [37]:
train = pd.read_csv('data/train.csv', encoding='utf-8')
train.dropna(inplace=True)
train = train[train.sentence_id != 492799]
train.reset_index(inplace=True)

#### Prepare full data

In [4]:
vectorizer = CountVectorizer (
    analyzer='char_wb',
    ngram_range=(1, 1),
    lowercase=False
)

In [5]:
%time data_ngrams = vectorizer.fit_transform(train['before'].values)

CPU times: user 1min 30s, sys: 1.87 s, total: 1min 32s
Wall time: 1min 33s


In [6]:
data_ngrams.shape

(10574501, 2660)

In [7]:
labels = np.array(train['after'].values != train['before'].values, dtype=np.uint8)

enc = LabelEncoder()
labels = enc.fit_transform(labels)

#### Prepare context data

In [38]:
def get_context_value(values, sentences, offset):
    offset_values = np.asarray([u'' for v in range(len(values))], dtype=object)
    for i in range(len(values)):
        offset_value = ""
        i_offset = i + offset
        if i_offset >= 0 and i_offset < len(values) \
                and sentences[i_offset] == sentences[i]:
            offset_value = values[i_offset]
        offset_values[i] = offset_value

    return offset_values

In [40]:
train['before_m_2'] = pd.Series(get_context_value(train.before.values, train.sentence_id.values, -2))
train['before_m_1'] = pd.Series(get_context_value(train.before.values, train.sentence_id.values, -1))
train['before_p_1'] = pd.Series(get_context_value(train.before.values, train.sentence_id.values, 1))
train['before_p_2'] = pd.Series(get_context_value(train.before.values, train.sentence_id.values, 2))

In [42]:
def generate_context_ngrams(data, columns):
    return scipy.sparse.hstack([vectorizer.transform(data[c].values) for c in columns])

In [43]:
context_cols = ['before_m_2', 'before_m_1', 'before', 'before_p_1', 'before_p_2']

#### Prepare train data

In [44]:
sample_sizes = {
    "PUNCT" : 25000,
    "PLAIN" : 30000,
    "CARDINAL" : 20000,
    "LETTERS" : 20000,
    "DATE" : 20000,
    "ORDINAL" : 15000,
    "MEASURE" : 15000
}

train.reset_index(inplace=True)
train_data = pd.DataFrame(columns=train.columns)
sample_max_size = 10000

for c in train['class'].unique():
    class_sample = train[train['class'] == c]
    sample_size = sample_sizes[c] if c in sample_sizes else sample_max_size
    if len(class_sample) > sample_max_size:
        train_data = train_data.append(class_sample.sample(sample_size))
    else:
        train_data = train_data.append(class_sample)

In [45]:
train_ngrams = generate_context_ngrams(train_data, context_cols)

In [46]:
labels_train = np.array(train_data['after'].values != train_data['before'].values, dtype=np.uint8)
labels_train = enc.transform(labels_train)

### Train XGBoost classifier 

In [47]:
xtr, xcv, ytr, ycv = train_test_split(train_ngrams, labels_train, test_size = 0.05, random_state = 42)
feature_names = ['f_{}'.format(i) for i in range(train_ngrams.shape[1])]

In [48]:
dtrain = xgb.DMatrix(xtr, label=ytr, feature_names=feature_names)
dvalid = xgb.DMatrix(xcv, label=ycv, feature_names=feature_names)

In [49]:
params = [("objective", "binary:logistic"),
          ("booster", "gbtree"),
          ("nthread", 3),
          ("eta", 0.01),
          ("max_depth", 8),
          ("subsample", 0.9),
          ("min_child_weight", 1),
          ("colsample_bytree", 0.7),
          ("eval_metric", 'error'),
          ("eval_metric", 'auc'),
         ]
num_rounds = 10000
stop = 60

In [50]:
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

In [51]:
gbm = xgb.train(params, dtrain, num_rounds, evals=watchlist, early_stopping_rounds=stop)

[0]	train-error:0.127067	train-auc:0.938747	valid-error:0.128285	valid-auc:0.938557
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 60 rounds.
[1]	train-error:0.095191	train-auc:0.959619	valid-error:0.094958	valid-auc:0.960181
[2]	train-error:0.090491	train-auc:0.964873	valid-error:0.089084	valid-auc:0.965013
[3]	train-error:0.079191	train-auc:0.976891	valid-error:0.077654	valid-auc:0.977523
[4]	train-error:0.07986	train-auc:0.981235	valid-error:0.077868	valid-auc:0.981791
[5]	train-error:0.071691	train-auc:0.98442	valid-error:0.071566	valid-auc:0.984667
[6]	train-error:0.073754	train-auc:0.98422	valid-error:0.073809	valid-auc:0.984462
[7]	train-error:0.068689	train-auc:0.985439	valid-error:0.068575	valid-auc:0.985504
[8]	train-error:0.067132	train-auc:0.985538	valid-error:0.066973	valid-auc:0.985509
[9]	train-error:0.073743	train-auc:0.985854	valid-error:0.0721	valid-auc:0.985919
[10]	train-error:0.067

[96]	train-error:0.060678	train-auc:0.992004	valid-error:0.061205	valid-auc:0.991726
[97]	train-error:0.060768	train-auc:0.992006	valid-error:0.061205	valid-auc:0.991713
[98]	train-error:0.060734	train-auc:0.992001	valid-error:0.060991	valid-auc:0.991654
[99]	train-error:0.060756	train-auc:0.992027	valid-error:0.061312	valid-auc:0.991723
[100]	train-error:0.060745	train-auc:0.992067	valid-error:0.061098	valid-auc:0.991781
[101]	train-error:0.060543	train-auc:0.992079	valid-error:0.060564	valid-auc:0.991814
[102]	train-error:0.060571	train-auc:0.992081	valid-error:0.060884	valid-auc:0.991802
[103]	train-error:0.060582	train-auc:0.992083	valid-error:0.060884	valid-auc:0.991826
[104]	train-error:0.060582	train-auc:0.992085	valid-error:0.060671	valid-auc:0.991816
[105]	train-error:0.06043	train-auc:0.992099	valid-error:0.060564	valid-auc:0.991837
[106]	train-error:0.060425	train-auc:0.992095	valid-error:0.06035	valid-auc:0.991809
[107]	train-error:0.060442	train-auc:0.992372	valid-error:0.

[192]	train-error:0.056079	train-auc:0.994686	valid-error:0.057573	valid-auc:0.994531
[193]	train-error:0.056023	train-auc:0.994699	valid-error:0.057466	valid-auc:0.994544
[194]	train-error:0.055545	train-auc:0.994833	valid-error:0.056932	valid-auc:0.994647
[195]	train-error:0.055545	train-auc:0.994823	valid-error:0.056932	valid-auc:0.994622
[196]	train-error:0.055579	train-auc:0.994798	valid-error:0.056932	valid-auc:0.994597
[197]	train-error:0.055472	train-auc:0.994846	valid-error:0.056825	valid-auc:0.994652
[198]	train-error:0.055427	train-auc:0.994834	valid-error:0.056719	valid-auc:0.994646
[199]	train-error:0.055309	train-auc:0.994849	valid-error:0.056719	valid-auc:0.994651
[200]	train-error:0.055331	train-auc:0.994869	valid-error:0.056825	valid-auc:0.994675
[201]	train-error:0.05523	train-auc:0.994951	valid-error:0.056612	valid-auc:0.994754
[202]	train-error:0.055219	train-auc:0.994954	valid-error:0.056612	valid-auc:0.994759
[203]	train-error:0.055196	train-auc:0.994949	valid-err

[288]	train-error:0.030404	train-auc:0.996035	valid-error:0.029801	valid-auc:0.995883
[289]	train-error:0.029414	train-auc:0.996055	valid-error:0.02852	valid-auc:0.995903
[290]	train-error:0.029144	train-auc:0.996063	valid-error:0.028413	valid-auc:0.995913
[291]	train-error:0.028689	train-auc:0.996075	valid-error:0.028092	valid-auc:0.995922
[292]	train-error:0.028363	train-auc:0.996091	valid-error:0.027665	valid-auc:0.995954
[293]	train-error:0.02834	train-auc:0.99609	valid-error:0.027665	valid-auc:0.995947
[294]	train-error:0.028284	train-auc:0.996093	valid-error:0.027451	valid-auc:0.995941
[295]	train-error:0.028273	train-auc:0.9961	valid-error:0.027451	valid-auc:0.995944
[296]	train-error:0.028627	train-auc:0.996134	valid-error:0.027558	valid-auc:0.995984
[297]	train-error:0.028323	train-auc:0.996131	valid-error:0.027451	valid-auc:0.995985
[298]	train-error:0.028323	train-auc:0.996149	valid-error:0.027451	valid-auc:0.995999
[299]	train-error:0.028267	train-auc:0.99615	valid-error:0.

[384]	train-error:0.02685	train-auc:0.996775	valid-error:0.025849	valid-auc:0.996621
[385]	train-error:0.026834	train-auc:0.996782	valid-error:0.025849	valid-auc:0.996629
[386]	train-error:0.026817	train-auc:0.996787	valid-error:0.025849	valid-auc:0.996633
[387]	train-error:0.026811	train-auc:0.996792	valid-error:0.025849	valid-auc:0.996639
[388]	train-error:0.026805	train-auc:0.996795	valid-error:0.025849	valid-auc:0.996648
[389]	train-error:0.025889	train-auc:0.996798	valid-error:0.024888	valid-auc:0.996652
[390]	train-error:0.025507	train-auc:0.996806	valid-error:0.023927	valid-auc:0.996656
[391]	train-error:0.025479	train-auc:0.996816	valid-error:0.023927	valid-auc:0.99666
[392]	train-error:0.025569	train-auc:0.996822	valid-error:0.024354	valid-auc:0.99667
[393]	train-error:0.025462	train-auc:0.996829	valid-error:0.023927	valid-auc:0.996679
[394]	train-error:0.025434	train-auc:0.996828	valid-error:0.023927	valid-auc:0.996678
[395]	train-error:0.025265	train-auc:0.996831	valid-error

...

[3336]	train-error:0.005442	train-auc:0.999611	valid-error:0.005448	valid-auc:0.999126
[3337]	train-error:0.005431	train-auc:0.999611	valid-error:0.005448	valid-auc:0.999126
[3338]	train-error:0.005425	train-auc:0.999611	valid-error:0.005448	valid-auc:0.999126
[3339]	train-error:0.005425	train-auc:0.999611	valid-error:0.005448	valid-auc:0.999126
[3340]	train-error:0.005425	train-auc:0.999611	valid-error:0.005448	valid-auc:0.999126
[3341]	train-error:0.005414	train-auc:0.999611	valid-error:0.005448	valid-auc:0.999126
[3342]	train-error:0.005414	train-auc:0.999612	valid-error:0.005448	valid-auc:0.999126
[3343]	train-error:0.005414	train-auc:0.999612	valid-error:0.005448	valid-auc:0.999126
[3344]	train-error:0.005408	train-auc:0.999612	valid-error:0.005448	valid-auc:0.999126
[3345]	train-error:0.005403	train-auc:0.999612	valid-error:0.005448	valid-auc:0.999127
[3346]	train-error:0.005403	train-auc:0.999612	valid-error:0.005448	valid-auc:0.999128
[3347]	train-error:0.005403	train-auc:0.999

[3431]	train-error:0.005251	train-auc:0.999624	valid-error:0.005234	valid-auc:0.999134
[3432]	train-error:0.005257	train-auc:0.999624	valid-error:0.005234	valid-auc:0.999134
[3433]	train-error:0.005245	train-auc:0.999624	valid-error:0.005234	valid-auc:0.999134
[3434]	train-error:0.005245	train-auc:0.999624	valid-error:0.005234	valid-auc:0.999134
[3435]	train-error:0.005245	train-auc:0.999624	valid-error:0.005234	valid-auc:0.999134
[3436]	train-error:0.005245	train-auc:0.999624	valid-error:0.005234	valid-auc:0.999133
[3437]	train-error:0.00524	train-auc:0.999625	valid-error:0.005234	valid-auc:0.999133
[3438]	train-error:0.005234	train-auc:0.999625	valid-error:0.005234	valid-auc:0.999133
[3439]	train-error:0.005234	train-auc:0.999625	valid-error:0.005234	valid-auc:0.999133
[3440]	train-error:0.005234	train-auc:0.999625	valid-error:0.005234	valid-auc:0.999133
[3441]	train-error:0.005234	train-auc:0.999625	valid-error:0.005234	valid-auc:0.999133
[3442]	train-error:0.005234	train-auc:0.9996

[3526]	train-error:0.005167	train-auc:0.999637	valid-error:0.005127	valid-auc:0.999138
[3527]	train-error:0.005167	train-auc:0.999637	valid-error:0.005127	valid-auc:0.999139
[3528]	train-error:0.005155	train-auc:0.999637	valid-error:0.005127	valid-auc:0.999139
[3529]	train-error:0.00515	train-auc:0.999637	valid-error:0.005127	valid-auc:0.999139
[3530]	train-error:0.005144	train-auc:0.999637	valid-error:0.005127	valid-auc:0.999139
[3531]	train-error:0.005138	train-auc:0.999637	valid-error:0.005127	valid-auc:0.999139
[3532]	train-error:0.005127	train-auc:0.999637	valid-error:0.005127	valid-auc:0.999139
[3533]	train-error:0.005127	train-auc:0.999638	valid-error:0.005127	valid-auc:0.99914
[3534]	train-error:0.005122	train-auc:0.999638	valid-error:0.005127	valid-auc:0.99914
[3535]	train-error:0.00511	train-auc:0.999638	valid-error:0.005127	valid-auc:0.99914
[3536]	train-error:0.00511	train-auc:0.999638	valid-error:0.005127	valid-auc:0.99914
[3537]	train-error:0.005105	train-auc:0.999639	val

[3621]	train-error:0.004998	train-auc:0.999651	valid-error:0.005127	valid-auc:0.999146
[3622]	train-error:0.004998	train-auc:0.999651	valid-error:0.005127	valid-auc:0.999146
[3623]	train-error:0.004992	train-auc:0.999651	valid-error:0.005127	valid-auc:0.999146
[3624]	train-error:0.004998	train-auc:0.999651	valid-error:0.005127	valid-auc:0.999146
[3625]	train-error:0.004998	train-auc:0.999651	valid-error:0.005127	valid-auc:0.999146
[3626]	train-error:0.004998	train-auc:0.999651	valid-error:0.005127	valid-auc:0.999147
[3627]	train-error:0.004998	train-auc:0.999651	valid-error:0.005127	valid-auc:0.999147
[3628]	train-error:0.004998	train-auc:0.999651	valid-error:0.005127	valid-auc:0.999147
[3629]	train-error:0.004998	train-auc:0.999651	valid-error:0.005127	valid-auc:0.999147
[3630]	train-error:0.004998	train-auc:0.999652	valid-error:0.005127	valid-auc:0.999147
[3631]	train-error:0.004992	train-auc:0.999652	valid-error:0.005127	valid-auc:0.999147
[3632]	train-error:0.004992	train-auc:0.999

[3716]	train-error:0.004857	train-auc:0.999661	valid-error:0.005127	valid-auc:0.999154
[3717]	train-error:0.004857	train-auc:0.999661	valid-error:0.005127	valid-auc:0.999154
[3718]	train-error:0.004857	train-auc:0.999662	valid-error:0.005127	valid-auc:0.999155
[3719]	train-error:0.004857	train-auc:0.999662	valid-error:0.005127	valid-auc:0.999155
[3720]	train-error:0.004852	train-auc:0.999662	valid-error:0.005127	valid-auc:0.999155
[3721]	train-error:0.004846	train-auc:0.999663	valid-error:0.005127	valid-auc:0.999154
[3722]	train-error:0.004846	train-auc:0.999663	valid-error:0.005127	valid-auc:0.999154
[3723]	train-error:0.004852	train-auc:0.999663	valid-error:0.005127	valid-auc:0.999154
[3724]	train-error:0.004852	train-auc:0.999663	valid-error:0.005127	valid-auc:0.999155
[3725]	train-error:0.004846	train-auc:0.999663	valid-error:0.005127	valid-auc:0.999155
[3726]	train-error:0.004846	train-auc:0.999663	valid-error:0.005127	valid-auc:0.999155
[3727]	train-error:0.004857	train-auc:0.999

In [52]:
print gbm.eval(dvalid)

[0]	eval-error:0.005127	eval-auc:0.999154


### Validate classification accuracy

In [53]:
%time data_ngrams = generate_context_ngrams(train, context_cols)

CPU times: user 7min 34s, sys: 23.7 s, total: 7min 57s
Wall time: 8min 2s


In [54]:
dtest = xgb.DMatrix(data_ngrams, label=labels, feature_names=feature_names)

In [55]:
%time predictions = gbm.predict(dtest)

CPU times: user 2h 49min 15s, sys: 14.2 s, total: 2h 49min 29s
Wall time: 1h 40s


In [56]:
predictions

array([ 0.01649959,  0.00064542,  0.00664725, ...,  0.008603  ,
        0.00384797,  0.00434028], dtype=float32)

In [57]:
from sklearn.metrics import accuracy_score

for i in range(10):
    predictions_current = predictions > i / 10.0
    print ("0.{}: {}".format(i, accuracy_score(labels, predictions_current)))

0.0: 0.125082403416
0.1: 0.954131736334
0.2: 0.975757343065
0.3: 0.985090927695
0.4: 0.988508204784
0.5: 0.992159630038
0.6: 0.993216133792
0.7: 0.993637619402
0.8: 0.993493971961
0.9: 0.991273630784


In [63]:
best_threshold = 0.7
best_accuracy = accuracy_score(labels, predictions > best_threshold)
print 'Best accuracy: ', best_accuracy
print 'Number of errors: ', len(train) - best_accuracy * len(train)

Best accuracy:  0.993637619402
Number of errors:  67279.0


In [60]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(labels, predictions > 0.7).ravel()

In [70]:
print tn, fp 
print fn, '\t', tp

9213176 38641
28638 	1294046
